In [1]:
import numpy as np

class AdaBoost:
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.alphas = []
        self.models = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        w = np.ones(n_samples) / n_samples

        for _ in range(self.n_estimators):
            stump = DecisionStump()
            stump.fit(X, y, sample_weights=w)

            y_pred = stump.predict(X)

            error = np.sum(w * (y_pred != y)) / np.sum(w)

            alpha = 0.5 * np.log((1 - error) / (error + 1e-10))

            w = w * np.exp(-alpha * y * y_pred)
            w = w / np.sum(w)  

            self.models.append(stump)
            self.alphas.append(alpha)

    def predict(self, X):
        predictions = np.zeros(X.shape[0])
        for alpha, model in zip(self.alphas, self.models):
            predictions += alpha * model.predict(X)
        return np.sign(predictions)

class DecisionStump:
    def fit(self, X, y, sample_weights=None):
        best_error = float('inf')
        best_stump = {}
        n_samples, n_features = X.shape

        if sample_weights is None:
            sample_weights = np.ones(n_samples) / n_samples

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                predictions = np.ones(n_samples)
                predictions[X[:, feature] < threshold] = -1

                error = np.sum(sample_weights * (predictions != y))

                if error < best_error:
                    best_error = error
                    best_stump = {
                        'feature': feature,
                        'threshold': threshold,
                        'predictions': predictions
                    }

        self.feature = best_stump['feature']
        self.threshold = best_stump['threshold']
        self.predictions = best_stump['predictions']

    def predict(self, X):
        predictions = np.ones(X.shape[0])
        predictions[X[:, self.feature] < self.threshold] = -1
        return predictions

if __name__ == "__main__":
    X = np.array([[1, 2], [2, 3], [3, 3], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9]])
    y = np.array([1, 1, -1, -1, 1, 1, -1, -1])

    adaboost = AdaBoost(n_estimators=50)
    adaboost.fit(X, y)

    predictions = adaboost.predict(X)
    print("Predictions:", predictions)

    accuracy = np.mean(predictions == y)
    print("Accuracy:", accuracy)

Predictions: [-1. -1. -1. -1. -1. -1. -1. -1.]
Accuracy: 0.5
